In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from gpt3forchem.data import get_photoswitch_data
from gpt3forchem.input import create_single_property_forward_prompts
from sklearn.model_selection import train_test_split
from gpt3forchem.api_wrappers import fine_tune, query_gpt3, extract_prediction, ensemble_fine_tune, multiple_query_gpt3
import time
from pycm import ConfusionMatrix
from gpt3forchem.baselines import GPRBaseline, compute_fragprints
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt

plt.style.use(["science", "nature"])


/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/gpflow/experimental/utils.py:42: UserWarning: You're calling gpflow.experimental.check_shapes.decorator.check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  warn(
/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/gpflow/experimental/utils.py:42: UserWarning: You're calling gpflow.experimental.check_shapes.inheritance.inherit_check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  warn(
/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = get_photoswitch_data()


In [4]:
prompts = create_single_property_forward_prompts(
    data,
    "wavelength_cat",
    {"wavelength_cat": "transition wavelength"},
    representation_col="SMILES",
)


In [5]:
train_prompts, test_prompts = train_test_split(
    prompts, test_size=0.2, random_state=None
)


In [12]:
models = ensemble_fine_tune(train_prompts, test_prompts, num_models=10)

Fine tuning on 10 train files and 10 valid files
Fine-tune ft-QbVNJZVajonZfMotpmnJRQGH has the status "pending" and will not be logged
🎉 wandb sync completed successfully
Uploaded file from run_files/2022-09-01-22-31-14_train__ensemble_3_250.jsonl: file-nkufd1b7hW7vFXB4LzQDC5Fo
Uploaded file from run_files/2022-09-01-22-31-14_valid__ensemble_3_79.jsonl: file-ewagjtWc7mHyeUEaQO3bLf8w
Created fine-tune: ft-ikYTFWBnrfvLkT3Id1fvHl4f
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-09-01 22:31:20] Created fine-tune: ft-ikYTFWBnrfvLkT3Id1fvHl4f

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-ikYTFWBnrfvLkT3Id1fvHl4f

 
Upload progress:   0%|          | 0.00/30.1k [00:00<?, ?it/s]
Upload progress: 100%|██████████| 30.1k/30.1k [00:00<00:00, 14.9Mit/s]

Upload progress:   0%|          | 0.00/9.52k [00:00<?, ?it/s]
Upload progress: 100%|██████████| 9.52k/9.52k [00:

In [10]:
models_0 = models

In [17]:
completions =   multiple_query_gpt3(models, test_prompts)

In [18]:
completions

[{'choices': [<OpenAIObject at 0x2967e40e0> JSON: {
     "finish_reason": "length",
     "index": 0,
     "logprobs": null,
     "text": " 2@@@@@@ 1@@@@@@"
   },
   <OpenAIObject at 0x2967d4e00> JSON: {
     "finish_reason": "length",
     "index": 1,
     "logprobs": null,
     "text": " 0@@@@@@@@@@@@@@"
   },
   <OpenAIObject at 0x2967d4ea0> JSON: {
     "finish_reason": "length",
     "index": 2,
     "logprobs": null,
     "text": " 2@@@@@@@@@@@@@@"
   },
   <OpenAIObject at 0x2967d4130> JSON: {
     "finish_reason": "length",
     "index": 3,
     "logprobs": null,
     "text": " 0@@@@@@@@@@@@@@"
   },
   <OpenAIObject at 0x2967d70e0> JSON: {
     "finish_reason": "length",
     "index": 4,
     "logprobs": null,
     "text": " 2@@@@@@@=@@@@"
   },
   <OpenAIObject at 0x2967d7090> JSON: {
     "finish_reason": "length",
     "index": 5,
     "logprobs": null,
     "text": " 0@@@@@@@@@@@@@@"
   },
   <OpenAIObject at 0x2967d72c0> JSON: {
     "finish_reason": "length",
     "index"

In [19]:
len(completions)

3

In [23]:
predictions = [
    [
        int(extract_prediction(completions[j], i))
        for i in range(len(completions[j]["choices"]))
    ]
    for j in range(len(completions))
]


In [33]:
pred = np.array(predictions).T

In [46]:
true = [
    int(test_prompts.iloc[i]["completion"].split("@")[0])
    for i in range(len(predictions[0]))
]

predictions are mostly the same but not always. Let's get the performance per prediction and for the majority vote.

In [34]:
maj_vote = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=1, arr=pred)


In [35]:
maj_vote

array([2, 0, 2, 0, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 0, 2,
       1, 2, 2, 0, 1, 2, 2, 0, 2, 0, 2, 0, 1, 0, 0, 2, 0, 0])

In [41]:
len(true)

3

In [49]:
cm_maj = ConfusionMatrix(maj_vote, true)

In [50]:
cm_0 = ConfusionMatrix(predictions[0], true)
cm_1 = ConfusionMatrix(predictions[1], true)
cm_2 = ConfusionMatrix(predictions[2], true)

In [52]:
print(cm_0.F1_Macro, cm_1.F1_Macro, cm_2.F1_Macro, cm_maj.F1_Macro)

0.35218390804597705 0.3517241379310345 0.5881481481481481 0.33766749379652605


In [53]:
print(cm_0.F1_Micro, cm_1.F1_Micro, cm_2.F1_Micro, cm_maj.F1_Micro)

0.575 0.575 0.675 0.575


In [54]:
print(cm_0.ACC_Macro, cm_1.ACC_Macro, cm_2.ACC_Macro, cm_maj.ACC_Macro)

0.8299999999999998 0.8299999999999998 0.8699999999999999 0.8299999999999998
